# Test build

In [1]:
import numpy as np
import os
import math
import time
import pandas as pd
import json
# NN
import torch
import torch.nn as nn
import torch.nn.functional as F
#from torch.utils.data import random_split
from torch.utils.data import DataLoader, Dataset
from torch import optim
from copy import deepcopy
from sklearn.model_selection import train_test_split
## Ploting
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
from matplotlib.colors import to_rgba
## Progress bar
from tqdm.notebook import tqdm
# Path
import sys
sys.path.append('/home/sebacastillo/neuralnets/')
from src.utils import get_project_root
root = get_project_root()
## Check torch version
print(f'Using {torch.__version__}')

device = torch.device('cuda' if torch.cuda.is_available() else torch.device('cpu'))
torch.manual_seed(42)
# GPU operations have a separate seed we also want to set
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Using 2.0.0+cu117


In [57]:
params = {
    "learning_rate": 0.001,
    "momentum": 0.6,
    "acc": 0.0,
    "epoca": 0,
    "input_file": '/data/XOR.csv',
    "EXP_NAME": 'EXP009',
    "MIN_ACC": 1.0,
    "MIN_ERROR": 1E6,
    "MAX_EPOCAS": 1000,
    "MAX_COUNTER": 50,
    "BATCH_SIZE": 68
}

Step by Step Train

Load data

In [58]:
def load_split_save_data(input_filename, output_name='EXP', split_type='train_test', train_ratio=0.75, validate_ratio=None, test_ratio=None):

    data = pd.read_csv(input_filename)

    # Check if 'exp' folder exists, create it if it doesn't
    if not os.path.exists('exp'):
        os.makedirs('exp')
    
    # Create a subfolder with the output_name
    output_path = os.path.join('exp', output_name)
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    if split_type == 'train_validate_test':
        if not validate_ratio or not test_ratio:
            raise ValueError("Please provide validate_ratio and test_ratio for 'train_validate_test' split type.")
        
        train_data, temp_data = train_test_split(data, train_size=train_ratio, random_state=42)
        validate_data, test_data = train_test_split(temp_data, train_size=validate_ratio / (validate_ratio + test_ratio), random_state=42)
        
        # Save the train, validate, and test data as CSV files in the output folder
        train_data.to_csv(os.path.join(output_path, f'{output_name}_train_data.csv'), index=False)
        validate_data.to_csv(os.path.join(output_path, f'{output_name}_validate_data.csv'), index=False)
        test_data.to_csv(os.path.join(output_path, f'{output_name}_test_data.csv'), index=False)


        return train_data, validate_data, test_data    

    elif split_type == 'train_test':
        train_data, test_data = train_test_split(data, train_size=train_ratio, random_state=42)
        
        # Save the train and test data as CSV files in the output folder
        train_data.to_csv(os.path.join(output_path, f'{output_name}_train_data.csv'), index=False)
        test_data.to_csv(os.path.join(output_path, f'{output_name}_test_data.csv'), index=False)


        return train_data, test_data
    
    else:
        raise ValueError("Invalid split_type. Use either 'train_validate_test' or 'train_test'.")



In [59]:
train, test = load_split_save_data((str(root) + '/data/XOR.csv'), 'EXP010', split_type='train_test')

In [60]:
class DATA(Dataset):
    '''
    Lee los datos 
    Alimenta modelos
    '''
    def __init__(self, data):
        data = data.to_numpy()
        self.X = data[:,:-1].astype(np.float32)
        y = data[:, -1].astype(np.float32)
        # Relabel class -1 to 0
        y[y== -1] = 0
        self.y = y
    
    def __len__(self):
        # devuelve numero observacines data
        return len(self.X)
    
    def __getitem__(self, idx):
        # devulve observaciones indexadas        
        return self.X[idx], self.y[idx]

In [61]:
train_data = DATA(train)
test_data = DATA(test)

In [62]:
train_loader = DataLoader(train_data, batch_size=params['BATCH_SIZE'], shuffle=True)
test_loader = DataLoader(test_data, batch_size=params['BATCH_SIZE'], shuffle=True)

Model

In [63]:
class MODEL(nn.Module):

    def __init__(self, n_features, n_inputs, n_outputs):
        
        super().__init__() # ejecuta init en nn.Module

        self.n_features = n_features
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs

        self.layer1 = nn.Linear(self.n_features, self.n_inputs, bias=True)
        self.layer2 = nn.Linear(self.n_inputs, self.n_outputs, bias=True)

        self.tanh = nn.Tanh()

    def forward(self, X):
        
        y_pred = self.layer1(X)
        y_pred = self.tanh(y_pred)
        y_pred = self.layer2(y_pred)
        return y_pred

Train and Test Steps

In [64]:
def train_step(model, data, loss_function, optimizer, device):

    model.train()

    N_batches = len(data)

    error = 0

    for idx,(X,y) in enumerate(data):

        X = X.to(device)
        y = y.to(device)
        
        optimizer.zero_grad() # volvemos a 0 weights

        y_pred = model(X)

        if (data.batch_size == 1):
            loss = loss_function(y_pred.squeeze(), y.squeeze())
        else:
            loss = loss_function(y_pred.squeeze(), y)
    
        error += loss.item()

        loss.backward()
        optimizer.step()
    
    error /= N_batches

    return error, model

In [65]:
def predict_step(model, data, loss_function, device):

    model.eval()

    N_batches = len(data)

    error = 0

    Y = torch.tensor([])
    Yp = torch.tensor([])

    with torch.no_grad():

        for idx, (X,y) in enumerate(data):

            Y = torch.hstack((Y, y.flatten()))
            
            X = X.to(device)
            y = y.to(device)

            y_pred = model(X)

            Yp = torch.hstack((Yp, y_pred.flatten().cpu()))

            loss = loss_function(y_pred.squeeze(), y.squeeze())

            error += loss.item()
    
    error /= N_batches

    return error, Y, Yp         


Experimento

In [66]:
#=============================================
# Inicializamos el modelo
#=============================================
modelo = MODEL(n_features=2, n_inputs=3, n_outputs=1)
#=============================================
# Definimos la función de LOSS a utilizar
#=============================================
loss_function = nn.BCEWithLogitsLoss(reduction='mean')

#=============================================
# Definimos el optimizador a utilizar
# >>> 3er paso del bacpropagation
#=============================================
optimizer = optim.SGD(modelo.parameters(), lr=params['learning_rate'], momentum=0.9)
#optimizer = optim.Adam(modelo.parameters(), lr=learning_rate)

In [ ]:
error = []  # Inicializo estructura para almacenar  los errores en el tiempo
accuracy = []  # Inicializo estructura para almacenar  el accuracy en el tiempo
STOP = False
counter = 0
best_model = None
best_model_weights = None
epoca = params['epoca']
acc = params['acc']

#===============================================================
while (epoca < params['MAX_EPOCAS']) and (acc < params['MIN_ACC']) and (not STOP):

    epoca += 1
    
    #----------------------
    # ENTRENAMIENTO
    #----------------------
    _,modelo = train_step(modelo, train_loader, loss_function, optimizer, device)
    
    #----------------------
    # VALIDACION
    #----------------------
    e,Y,Yp = predict_step(modelo, test_loader, loss_function, device)
    
    # TRANSFORMO SALIDA EN {0,1}
    Y_pred = torch.sigmoid(Yp)
    Y_pred[Y_pred < 0.5] = 0
    Y_pred[Y_pred > 0] = 1
    acc = torch.sum(Y_pred == Y)/ len(Y)
    
    #----------------------
    # ALMACENO MEDIDAS
    #----------------------
    error.append(e)
    accuracy.append(acc)
    
    #-----------------------------------------------
    # CRITERIO DE CORTE Y ALMACENAMIENTO DEL MODELO
    #-----------------------------------------------
    if (e < params['MIN_ERROR']):
        params['MIN_ERROR'] = e
        counter = 0
        
        #·······················
        # Almaceno el modelo
        #·······················
        best_model = deepcopy(modelo)  # Genero una copia independiente
        best_model_weights = best_model.state_dict()
        
    else:
        counter += 1
        if counter > params['MAX_COUNTER']:
            STOP = True
    
    #--------------------------------------------
    # MUESTRO REPORTE POR PANTALLA (POR EPOCA)
    #--------------------------------------------
    if (epoca % 10) == 0:
        print(f'Epoca: {epoca} -- Error: {e:.4}\t--\tTasa acierto [train]: {acc}\n')
#===============================================================

#--------------------------------------------
# MUESTRO REPORTE POR PANTALLA (FINAL)
#--------------------------------------------
print('='*79)
print(f'FINAL -- Epoca: {epoca} -- Error: {e:.4}\t--\tTasa acierto [train]: {acc:.4}')
print('='*79)

#-----------------------------
# GUARDO MEJOR MODELO A DISCO
#-----------------------------
torch.save(best_model,
           'best_model.pt',
           _use_new_zipfile_serialization=True)
        
#----------------------------------------------
# GUARDAMOS LOS PESOS DEL MEJOR MODELO A DISCO
#----------------------------------------------
torch.save(best_model.state_dict(),
           'best_model_state_dict.pt',
           _use_new_zipfile_serialization=True)